# Rotación de imágenes

## Descripción matemática

Para rotar una imagen un ángulo θ (en grados) alrededor de su centro, típicamente se siguen estos pasos matemáticos:

1. **Conversión de grados a radianes**  
   $$
   \theta_r = \theta \times \frac{\pi}{180}
   $$  
   donde θₖ es el ángulo en radianes que usarás en las funciones trigonométricas.

2. **Determinar el centro de rotación**  
   Si la imagen tiene ancho $W$ y alto $H$, su centro en coordenadas de píxel (suponiendo origen en la esquina superior izquierda y ejes positivos hacia la derecha y hacia abajo) es  
   $$
   (c_x, c_y) = \bigl(\tfrac{W-1}{2},\,\tfrac{H-1}{2}\bigr).
   $$

3. **Formar la matriz de rotación**  
   En coordenadas cartesianas con origen en el punto que quieras (aún no trasladado), la rotación se expresa con  
   $$
   R = \begin{pmatrix}
     \cos\theta_r & -\sin\theta_r \\[6pt]
     \sin\theta_r & \;\cos\theta_r
   \end{pmatrix}.
   $$

4. **Traslación para centrar la imagen en el origen**  
   Para rotar alrededor de $(c_x,c_y)$, primero trasladas cada punto $(x,y)$ para que el centro quede en $(0,0)$:
   $$
   \begin{pmatrix} x_0 \\ y_0 \end{pmatrix}
   =
   \begin{pmatrix} x - c_x \\ y - c_y \end{pmatrix}.
   $$

5. **Aplicar la rotación**  
   Multiplicas por $R$:
   $$
   \begin{pmatrix} x_1 \\ y_1 \end{pmatrix}
   = R \;\begin{pmatrix} x_0 \\ y_0 \end{pmatrix}
   = \begin{pmatrix}
     x_0 \cos\theta_r - y_0 \sin\theta_r \\[4pt]
     x_0 \sin\theta_r + y_0 \cos\theta_r
   \end{pmatrix}.
   $$

6. **Trasladar de nuevo al sistema de coordenadas de la imagen**  
   $$
   \begin{pmatrix} x' \\ y' \end{pmatrix}
   =
   \begin{pmatrix} x_1 + c_x \\ y_1 + c_y \end{pmatrix}.
   $$

7. **Mapeo inverso y cálculo del tamaño del lienzo resultante**  
   - Para evitar “vacíos” en la imagen de salida, se calcula el **bounding box** de las cuatro esquinas rotadas y se ajusta el ancho y alto del lienzo nuevo.  
   - En la práctica de procesamiento de imágenes, por cada píxel $(i,j)$ de la imagen destino se aplica el **mapeo inverso** (usando $-\theta$) para encontrar de dónde “toma” el valor en la imagen original:
     $$
     \begin{pmatrix} x \\ y \end{pmatrix}
     =
     R(-\theta_r)\;\Bigl(\begin{pmatrix} i \\ j \end{pmatrix} - \begin{pmatrix} c_x \\ c_y \end{pmatrix}\Bigr)
     + \begin{pmatrix} c_x \\ c_y \end{pmatrix}.
     $$
   - Dado que $(x,y)$ casi nunca serán enteros, se interpola (p. ej. bilineal) para obtener el valor de píxel.

8. **Interpolación de píxel**  
   - **Nearest–neighbor**: redondeas $(x,y)$ al entero más cercano.  
   - **Bilineal**: pesas los cuatro píxeles circundantes según las distancias fraccionales en $x$ y $y$.  
   - **Bicúbica** u otros métodos más avanzados pueden usarse para mejor calidad.

9. **Construcción del resultado**  
   Iteras sobre cada píxel de la imagen de salida, aplicas el mapeo inverso, interpolas, y asignas el valor resultante. De este modo evitas huecos y artefactos.

---

**Resumen matricial (homogénea)**  
A menudo se combina todo en una sola matriz $3\times3$ usando coordenadas homogéneas, lo cual permite concatenar fácilmente traslaciones y rotaciones:

$$
T_{\text{centro}} = 
\begin{pmatrix}
1 & 0 & -c_x\\
0 & 1 & -c_y\\
0 & 0 & 1
\end{pmatrix},
\quad
R_h =
\begin{pmatrix}
\cos\theta_r & -\sin\theta_r & 0\\
\sin\theta_r & \;\cos\theta_r & 0\\
0 & 0 & 1
\end{pmatrix},
\quad
T_{\text{regreso}} =
\begin{pmatrix}
1 & 0 & c_x\\
0 & 1 & c_y\\
0 & 0 & 1
\end{pmatrix}.
$$

La transformación completa es  
$$
M = T_{\text{regreso}}\;R_h\;T_{\text{centro}}
$$
y un punto $\begin{pmatrix}x\\y\\1\end{pmatrix}$ se transforma como $\;M\begin{pmatrix}x\\y\\1\end{pmatrix}.$